In [1]:
import pywt

## ウェーブレットパケット構造

- ウェーブレットパケットを生成してみる。

In [2]:
x = [1, 2, 3 , 4, 5, 6, 7, 8]
wp = pywt.WaveletPacket(data=x, wavelet="db1", mode="symmetric")

- 入力データと各係数は、 `WaveletPacket.data` の属性で保持されている。

In [3]:
print(wp.data)

[1, 2, 3, 4, 5, 6, 7, 8]


- このオブジェクトからは、ウェーブレットパケットツリーにおけるノード(Node)を取得することができる。
- また、ツリーの深さに照応する水準(level)も確認することができる。

In [4]:
print(repr(wp.path))

''


- 返り値が `""` なのは、分解の水準が  `0` であるためである。実際水準は以下のように確認できる。

In [5]:
print(wp.level)

0


- 水準の最大値は、特にパラメタで指定していない場合は、自動的に計算される。その結果は `maxlevel` の属性で保持されている。

In [6]:
print(wp["ad"].maxlevel)

3


- 離散ウェーブレット変換の定式を前提とするなら、上記のキー名となっている `ad` は、恐らく近似(Approximation)と詳細(Detail)のそれぞれを表しているとしか推理できない。
- これは後述するサブノートへのアクセスの処理で明示的に言及されるが、[API仕様](https://pywavelets.readthedocs.io/en/latest/ref/wavelet-packets.html#waveletpacket-and-waveletpacket-tree-node)と照合してみても、現時点でこの解釈には矛盾が見受けられない。

## ツリーの横断

- ウェーブレットパケットツリーの各水準における各ノードの情報や、ツリーの構造に関わる情報を順次確認していく。

### サブノートへのアクセス

- ウェーブレットパケットツリーのサブノートへのアクセスを試みる。
- どうやら、 `a` と `d` の組み合わせから成る任意の長さの文字列でツリーへのパスを指定するらしい。

#### 第一水準

In [7]:
print(wp["a"].data)

[  2.12132034   4.94974747   7.77817459  10.60660172]


In [8]:
print(wp["a"].path)

a


In [9]:
print(wp["d"].data)

[-0.70710678 -0.70710678 -0.70710678 -0.70710678]


In [10]:
print(wp["d"].path)

d


#### 第二水準

In [11]:
print(wp["aa"].data)

[  5.  13.]


In [13]:
print(wp["aa"].path)

aa


In [14]:
print(wp["ad"].data)

[-2. -2.]


In [15]:
print(wp["ad"].path)

ad


In [16]:
print(wp["dd"].data)

[ -1.11022302e-16   0.00000000e+00]


In [17]:
print(wp["dd"].path)

dd


In [18]:
print(wp["da"].data)

[-1. -1.]


In [19]:
print(wp["da"].path)

da


#### 第三水準

In [21]:
print(wp["aaa"].data)
print(wp["aaa"].path)

[ 12.72792206]
aaa


#### 例外の水準

- `maxlevel` を超過した水準を指定すると、 `IndexError` の例外がスローされる。


In [22]:
print(wp["aaaa"].data)

IndexError: Path length is out of range.

- 尚、 `a` や `d` 以外の文字列でパスを指定しようとすると、 `ValueError` の例外がスローされる。

In [23]:
print(wp["ac"].data)

ValueError: Subnode name must be in ['a', 'd'], not 'c'.

### ノードの属性へのアクセス

- `WaveletPacket` オブジェクトはツリー状のデータ構造である。
- このデータ構造は、各 `Node` オブジェクトの集合として評価される。
- 論理的には、`WaveletPacket` は、`Node` クラスのサブクラスである。
    * ちなみに `Node` クラスは `BaseNode` クラスを継承している。


- それぞれのノードオブジェクトは、先に取り上げた `data` や `path` や `maxlevel` の他に、 `node_name` や `parent` や `level` などの属性を保持している。

In [25]:
print(wp["ad"].node_name)
print(wp["ad"].parent)
print(wp["ad"].level)

d
a: [  2.12132034   4.94974747   7.77817459  10.60660172]
a
2


- `parent.path` を参照すれば、親ノードのパスも得られる。

In [27]:
print(wp["aad"].parent.path)

aa


- 離散ウェーブレット変換と同じように、 `mode` も得られる。

In [29]:
print(wp["ad"].mode)

symmetric


### ノードの収集

- 実装水準では、パスを明示的に指定するのみならず、全てのパスを逐次参照するというユースケースもあり得る。
- このライブラリでは、ツリー状のデータ構造のノードをどのような順序で参照していくのかを、一定の仕様で規定している。
- 以下は `natural` でソートした場合である。


In [30]:
print([node.path for node in wp.get_level(3, "natural")])

['aaa', 'aad', 'ada', 'add', 'daa', 'dad', 'dda', 'ddd']


- 一方、以下は帯域周波数でソートした場合である。

In [31]:
print([node.path for node in wp.get_level(3, "freq")])

['aaa', 'aad', 'add', 'ada', 'dda', 'ddd', 'dad', 'daa']


- 尚、 `get_level` メソッドは、その第一引数である `level` で指定した数値の水準に至るまで、自動的に分解を実行する。

### ウェーブレットパケットからのデータの再構成

- 新たに `WaveletPacket` オブジェクトを生成し、幾つかのデータをセットする。

In [32]:
new_wp = pywt.WaveletPacket(data=None, wavelet="db1", mode="symmetric")
new_wp["aa"] = wp["aa"].data
new_wp["ad"] = [-2., -2.]

- 利便性の観点から、 `Node.data` は `Node` のオブジェクトから自動的に抽出されるようにしておく。

In [33]:
new_wp["d"] = wp["d"]

- そして、`aa`、 `ad`、 `d` からデータを再構成する。

In [34]:
print(new_wp.reconstruct(update=False))

[ 1.  2.  3.  4.  5.  6.  7.  8.]


- もし `reconstruct` メソッドの引数となる `update` に `False` を指定したなら、 `data` の値は変わらない。

In [35]:
print(new_wp.data)

None


- 反面、同引数に `True` を指定した場合、 `data` の値は文字通り更新される。

In [36]:
print(new_wp.reconstruct(update=True))
print(new_wp.data)

[ 1.  2.  3.  4.  5.  6.  7.  8.]
[ 1.  2.  3.  4.  5.  6.  7.  8.]


# 参考資料

- [Wavelet Packets &#8212; PyWavelets Documentation](https://pywavelets.readthedocs.io/en/latest/regression/wp.html) (アクセス日時: 2018/01/13 14:20)